In [1]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages

In [2]:
ClIENT_ID = 'WTYXTX1VKJMLXOJPLAD1IJKLASAMY20FDVGFA1Z1X4XTYAHF' # your Foursquare ID
ClIENT_SECRET = 'XV3U42TL5SPPKSLA4KYBIKK3JW3FRLEJDZJJNMQCXS31ZY5J' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: WTYXTX1VKJMLXOJPLAD1IJKLASAMY20FDVGFA1Z1X4XTYAHF
Foursquare_Secret:XV3U42TL5SPPKSLA4KYBIKK3JW3FRLEJDZJJNMQCXS31ZY5J


In [3]:
city = 'bangalore'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

12.9791198 77.5912997


## Hotels

In [4]:
search_query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WTYXTX1VKJMLXOJPLAD1IJKLASAMY20FDVGFA1Z1X4XTYAHF&client_secret=XV3U42TL5SPPKSLA4KYBIKK3JW3FRLEJDZJJNMQCXS31ZY5J&ll=12.9791198,77.5912997&v=20180604&query=Hotel&radius=500&limit=30'

In [5]:
results = requests.get(url).json()


In [6]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ff931735', 'name': 'C...",False,53940333498e8a555ae0e2f3,NaN,IN,Bangalore,India,468,"[Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.98220710849808...",12.982207,77.594229,Vasanth Nagar,NaN,Karnātaka,Capitol Hotel,v-1566573233
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4f3bae47e4b04caf00fa4164,NaN,IN,Bangalore,India,461,"[Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.98209185115043...",12.982092,77.594259,NaN,NaN,Karnātaka,Hotel Paraag,v-1566573233
2,[],False,4de3d2418877bcb68664fc01,Bandra Kurla Complex,IN,Mumbai,India,513,"[Bandra Kurla Complex, Mumbai, Maharastra, India]","[{'label': 'display', 'lat': 12.983711, 'lng':...",12.983711,77.591756,NaN,NaN,Maharastra,Trident Hotel,v-1566573233
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4bc7779f93bdeee1a66a37ae,"3, Raj Bhavan Rd",IN,Bangalore,India,476,"[3, Raj Bhavan Rd, Bangalore 560001, Karnātaka...","[{'label': 'display', 'lat': 12.98198012398084...",12.981980,77.594564,NaN,560001,Karnātaka,24 Carats,v-1566573233


### keep only columns that include venue name, and anything that is associated with location


In [7]:
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Capitol Hotel,Convention Center,NaN,IN,Bangalore,India,468,"[Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.98220710849808...",12.982207,77.594229,Vasanth Nagar,NaN,Karnātaka,53940333498e8a555ae0e2f3
1,Hotel Paraag,Hotel,NaN,IN,Bangalore,India,461,"[Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.98209185115043...",12.982092,77.594259,NaN,NaN,Karnātaka,4f3bae47e4b04caf00fa4164
2,Trident Hotel,None,Bandra Kurla Complex,IN,Mumbai,India,513,"[Bandra Kurla Complex, Mumbai, Maharastra, India]","[{'label': 'display', 'lat': 12.983711, 'lng':...",12.983711,77.591756,NaN,NaN,Maharastra,4de3d2418877bcb68664fc01
3,24 Carats,Hotel,"3, Raj Bhavan Rd",IN,Bangalore,India,476,"[3, Raj Bhavan Rd, Bangalore 560001, Karnātaka...","[{'label': 'display', 'lat': 12.98198012398084...",12.981980,77.594564,NaN,560001,Karnātaka,4bc7779f93bdeee1a66a37ae


In [9]:
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Capitol Hotel,Convention Center,NaN,12.982207,77.594229,NaN,Karnātaka
1,Hotel Paraag,Hotel,NaN,12.982092,77.594259,NaN,Karnātaka
2,Trident Hotel,None,Bandra Kurla Complex,12.983711,77.591756,NaN,Maharastra
3,24 Carats,Hotel,"3, Raj Bhavan Rd",12.981980,77.594564,560001,Karnātaka


In [10]:
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,state
3,24 Carats,Hotel,"3, Raj Bhavan Rd",12.98198,77.594564,560001,Karnātaka


In [11]:
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
3,24 Carats,Hotel,"3, Raj Bhavan Rd",12.98198,77.594564,560001,Karnātaka


In [12]:
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels

,name,categories,address,lat,lng,postalCode,state
3,24 Carats,Hotel,"3, Raj Bhavan Rd",12.98198,77.594564,560001,Karnātaka


In [13]:
df_hotel = df_hotels[df_hotels.postalCode == '560001']
df_hotel

,name,categories,address,lat,lng,postalCode,state
3,24 Carats,Hotel,"3, Raj Bhavan Rd",12.98198,77.594564,560001,Karnātaka


## Paying Guest locations

In [14]:
search_query = 'Paying Guest'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WTYXTX1VKJMLXOJPLAD1IJKLASAMY20FDVGFA1Z1X4XTYAHF&client_secret=XV3U42TL5SPPKSLA4KYBIKK3JW3FRLEJDZJJNMQCXS31ZY5J&ll=12.9791198,77.5912997&v=20180604&query=Paying Guest&radius=10000&limit=30'

In [15]:
Rresults = requests.get(url).json()


In [20]:
venues = Rresults['response']['venues']

# tranform venues into a dataframe
PG_dataframe = json_normalize(venues)
PG_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '5032891291d4c4b30a586d68', 'name': 'A...",False,52cc311d498ee7c1f56e5338,NaN,IN,NaN,India,NaN,5657,[India],"[{'label': 'display', 'lat': 12.92832652033343...",12.928327,77.593112,NaN,NaN,NaN,Paying Guest,v-1566573415
1,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",False,5192a94a498e986eab599eb8,"5th Main, 5th Cross",IN,Bangalore,India,BTM 2nd Stage,7334,"[5th Main, 5th Cross (BTM 2nd Stage), Bangalor...","[{'label': 'display', 'lat': 12.91471930540871...",12.914719,77.605568,NaN,560076,Karnātaka,Paying Guest,v-1566573415
2,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",False,5141d948e4b0fc0922f785b4,#60,IN,Bangalore,India,"4th Cross, Kanaka Nagar",6389,"[#60 (4th Cross, Kanaka Nagar), Bangalore 5600...","[{'label': 'display', 'lat': 13.03421977046447...",13.034220,77.607800,NaN,560032,Karnātaka,Paying Guest Accommodation,v-1566573415
3,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",False,4fc377c4e4b06a3981d13063,Maruthi Layout,IN,Banglore,India,3rd Main,6784,"[Maruthi Layout (3rd Main), Banglore, Karnātak...","[{'label': 'display', 'lat': 12.92038154602050...",12.920382,77.607986,NaN,NaN,Karnātaka,Paying Guest Hostel(PG),v-1566573415
4,"[{'id': '4bf58dd8d48988d1ee931735', 'name': 'H...",False,539d8f88498e0d932e81c277,NaN,IN,NaN,India,NaN,4749,[India],"[{'label': 'display', 'lat': 12.93967698139066...",12.939677,77.608002,NaN,NaN,NaN,Evergreen Paying Guest,v-1566573415


In [21]:
PG_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_PG_dataframe = PG_dataframe.loc[:,PG_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_PG_dataframe['categories'] = clean_PG_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_PG_dataframe.columns = [column.split('.')[-1] for column in clean_PG_dataframe.columns]

clean_PG_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Paying Guest,Assisted Living,NaN,IN,NaN,India,NaN,5657,[India],"[{'label': 'display', 'lat': 12.92832652033343...",12.928327,77.593112,NaN,NaN,NaN,52cc311d498ee7c1f56e5338
1,Paying Guest,Residential Building (Apartment / Condo),"5th Main, 5th Cross",IN,Bangalore,India,BTM 2nd Stage,7334,"[5th Main, 5th Cross (BTM 2nd Stage), Bangalor...","[{'label': 'display', 'lat': 12.91471930540871...",12.914719,77.605568,NaN,560076,Karnātaka,5192a94a498e986eab599eb8
2,Paying Guest Accommodation,Residential Building (Apartment / Condo),#60,IN,Bangalore,India,"4th Cross, Kanaka Nagar",6389,"[#60 (4th Cross, Kanaka Nagar), Bangalore 5600...","[{'label': 'display', 'lat': 13.03421977046447...",13.034220,77.607800,NaN,560032,Karnātaka,5141d948e4b0fc0922f785b4
3,Paying Guest Hostel(PG),Residential Building (Apartment / Condo),Maruthi Layout,IN,Banglore,India,3rd Main,6784,"[Maruthi Layout (3rd Main), Banglore, Karnātak...","[{'label': 'display', 'lat': 12.92038154602050...",12.920382,77.607986,NaN,NaN,Karnātaka,4fc377c4e4b06a3981d13063
4,Evergreen Paying Guest,Hostel,NaN,IN,NaN,India,NaN,4749,[India],"[{'label': 'display', 'lat': 12.93967698139066...",12.939677,77.608002,NaN,NaN,NaN,539d8f88498e0d932e81c277


In [22]:
clean_PG_dataframe2= clean_PG_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_PG_dataframe2

,name,categories,address,crossStreet,lat,lng,postalCode,state
0,Paying Guest,Assisted Living,NaN,NaN,12.928327,77.593112,NaN,NaN
1,Paying Guest,Residential Building (Apartment / Condo),"5th Main, 5th Cross",BTM 2nd Stage,12.914719,77.605568,560076,Karnātaka
2,Paying Guest Accommodation,Residential Building (Apartment / Condo),#60,"4th Cross, Kanaka Nagar",13.034220,77.607800,560032,Karnātaka
3,Paying Guest Hostel(PG),Residential Building (Apartment / Condo),Maruthi Layout,3rd Main,12.920382,77.607986,NaN,Karnātaka
4,Evergreen Paying Guest,Hostel,NaN,NaN,12.939677,77.608002,NaN,NaN
5,Sanjay paying guest,Assisted Living,2nd House on the right,"11th cross, 11th main, 80ft road, Indiranagar",12.971048,77.647066,NaN,Karnātaka
6,FRIENDS Paying Guest,Assisted Living,Kormangala,"Green Leaf Layout, 4th Block",12.935321,77.629585,NaN,Karnātaka
7,Sunrise Paying Guest,Hostel,30-103/1,"Hennur 3rd cross, Kalyan Nagar",13.029033,77.635450,560043,Karnātaka
8,atithishala Paying Guest,Residential Building (Apartment / Condo),342 6th Cross KR Layout,6th Phase JP Nagar,12.903570,77.581674,560078,Karnātaka
9,SLNS Paying Guest PG,None,NaN,NaN,12.921725,77.663786,NaN,Karnātaka


In [23]:
df_PG = clean_PG_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_PG

,name,categories,address,crossStreet,lat,lng,postalCode,state
1,Paying Guest,Residential Building (Apartment / Condo),"5th Main, 5th Cross",BTM 2nd Stage,12.914719,77.605568,560076,Karnātaka
2,Paying Guest Accommodation,Residential Building (Apartment / Condo),#60,"4th Cross, Kanaka Nagar",13.034220,77.607800,560032,Karnātaka
7,Sunrise Paying Guest,Hostel,30-103/1,"Hennur 3rd cross, Kalyan Nagar",13.029033,77.635450,560043,Karnātaka
8,atithishala Paying Guest,Residential Building (Apartment / Condo),342 6th Cross KR Layout,6th Phase JP Nagar,12.903570,77.581674,560078,Karnātaka
16,Lic Vip Guesthouse,Boarding House,"16, Raja Ram Mohan Roy Road,",Sampangi Rama Nagara,12.966786,77.595310,560027,Karnātaka
17,Cosmos Guest Homes,Motel,"No 2814, Hal 2nd Stage, 18th A Main","5th B Cross, Old Airport Rd",12.974301,77.645455,560008,Karnātaka


## TechParks

In [24]:
search_query = 'Park'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WTYXTX1VKJMLXOJPLAD1IJKLASAMY20FDVGFA1Z1X4XTYAHF&client_secret=XV3U42TL5SPPKSLA4KYBIKK3JW3FRLEJDZJJNMQCXS31ZY5J&ll=12.9791198,77.5912997&v=20180604&query=Park&radius=10000&limit=30'

In [25]:
presults = requests.get(url).json()


In [26]:
venues = presults['response']['venues']

# tranform venues into a dataframe
park_dataframe = json_normalize(venues)
park_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4bbc4513e45295217db855a4,M G Road,IN,Bangalore,India,NaN,489,"[M G Road, Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.97704217520378...",12.977042,77.595277,NaN,Karnātaka,Cubbon Park,v-1566573826
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ca83e2ca6e08cfa26708d94,Near Anand Rao Circle,IN,Bangalore,India,"5/43 , Race Course Road",1657,"[Near Anand Rao Circle (5/43 , Race Course Roa...","[{'label': 'display', 'lat': 12.98124239713509...",12.981242,77.576172,NaN,Karnātaka,Fortune Park JP Celestial,v-1566573826
2,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4f105c6fe4b072565b7811fa,NaN,IN,NaN,India,NaN,2042,[India],"[{'label': 'display', 'lat': 12.99253763031361...",12.992538,77.578450,NaN,NaN,Kumara Park West Park,v-1566573826
3,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",False,4d88a92526a36ea88c04d0ad,"4/1, Bannerghatta Road",IN,Bangalore,India,NaN,5304,"[4/1, Bannerghatta Road, Bangalore 560029, Kar...","[{'label': 'display', 'lat': 12.93283615034002...",12.932836,77.602914,560029,Karnātaka,IBC Knowledge Park,v-1566573826
4,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4d181689512f2c0f952036cc,Rest House Road,IN,Bangalore,India,NaN,1659,"[Rest House Road, Bangalore 560001, Karnātaka,...","[{'label': 'display', 'lat': 12.97269293084298...",12.972693,77.605104,560001,Karnātaka,Rest House Park,v-1566573826


In [27]:
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Cubbon Park,Park,M G Road,IN,Bangalore,India,NaN,489,"[M G Road, Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.97704217520378...",12.977042,77.595277,NaN,Karnātaka,4bbc4513e45295217db855a4
1,Fortune Park JP Celestial,Hotel,Near Anand Rao Circle,IN,Bangalore,India,"5/43 , Race Course Road",1657,"[Near Anand Rao Circle (5/43 , Race Course Roa...","[{'label': 'display', 'lat': 12.98124239713509...",12.981242,77.576172,NaN,Karnātaka,4ca83e2ca6e08cfa26708d94
2,Kumara Park West Park,Park,NaN,IN,NaN,India,NaN,2042,[India],"[{'label': 'display', 'lat': 12.99253763031361...",12.992538,77.578450,NaN,NaN,4f105c6fe4b072565b7811fa
3,IBC Knowledge Park,Building,"4/1, Bannerghatta Road",IN,Bangalore,India,NaN,5304,"[4/1, Bannerghatta Road, Bangalore 560029, Kar...","[{'label': 'display', 'lat': 12.93283615034002...",12.932836,77.602914,560029,Karnātaka,4d88a92526a36ea88c04d0ad
4,Rest House Park,Park,Rest House Road,IN,Bangalore,India,NaN,1659,"[Rest House Road, Bangalore 560001, Karnātaka,...","[{'label': 'display', 'lat': 12.97269293084298...",12.972693,77.605104,560001,Karnātaka,4d181689512f2c0f952036cc


In [28]:
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe2

,name,categories,address,crossStreet,lat,lng,postalCode,state
0,Cubbon Park,Park,M G Road,NaN,12.977042,77.595277,NaN,Karnātaka
1,Fortune Park JP Celestial,Hotel,Near Anand Rao Circle,"5/43 , Race Course Road",12.981242,77.576172,NaN,Karnātaka
2,Kumara Park West Park,Park,NaN,NaN,12.992538,77.578450,NaN,NaN
3,IBC Knowledge Park,Building,"4/1, Bannerghatta Road",NaN,12.932836,77.602914,560029,Karnātaka
4,Rest House Park,Park,Rest House Road,NaN,12.972693,77.605104,560001,Karnātaka
5,Cubbon Park- Dog Park,Dog Run,NaN,NaN,12.974515,77.590203,NaN,NaN
6,The Park Hotel,Hotel,14/7 Mahatma Gandhi Road,NaN,12.973739,77.619766,NaN,Karnātaka
7,Eden Park Restaurant,Indian Restaurant,"Next to Hindu Office, Off Infantry Road",NaN,12.983456,77.598707,560052,Karnātaka
8,Hotel Park Residency,Bed & Breakfast,NaN,NaN,12.972849,77.608019,NaN,NaN
9,Cubbon Park Metro Station,Metro Station,Cubbon Park,NaN,12.980437,77.597878,NaN,Karnātaka


In [29]:
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3

,name,categories,address,crossStreet,lat,lng,postalCode,state
10,Manyata Embassy Business Park,Corporate Amenity,"Outer Ring Road, Nagavara",Nagavara,13.041540,77.619803,560 045,Karnātaka
18,Bruhat Bangalore Mahanagar Palike Park,Park,Indiranagar,1st Stage,12.982505,77.594327,560038,Karnātaka
22,Cariappa Park,Park,M G Road,Cubbon Road,12.976940,77.602032,560001,Karnātaka
23,Freedom Park,Park,Seshadri Road,Seshadri Road,12.978008,77.582373,560001,Karnātaka
27,Prestige Park Square,Real Estate Office,"Sy. No. 25, Bannerghatta Road, Near NICE Road ...",https://www.prestigeparksquarebangalore.ind.in,12.972358,77.594150,560083,Karnātaka


In [31]:
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Corporate Amenity']
df_park

,name,categories,address,crossStreet,lat,lng,postalCode,state
10,Manyata Embassy Business Park,Corporate Amenity,"Outer Ring Road, Nagavara",Nagavara,13.04154,77.619803,560 045,Karnātaka


## Restaurants

In [32]:
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WTYXTX1VKJMLXOJPLAD1IJKLASAMY20FDVGFA1Z1X4XTYAHF&client_secret=XV3U42TL5SPPKSLA4KYBIKK3JW3FRLEJDZJJNMQCXS31ZY5J&ll=12.9791198,77.5912997&v=20180604&query=Restaurant&radius=10000&limit=30'

In [33]:
Rresults = requests.get(url).json()

In [34]:
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4b6ee375f964a52029cf2ce3,"402-A, Millers Road, Vasanthnagar,",IN,Bangalore,India,NaN,1504,"[402-A, Millers Road, Vasanthnagar,, Bangalore...","[{'label': 'display', 'lat': 12.99227960861634...",12.992280,77.594473,560052,Karnātaka,Ujwal Bar & Restaurant,v-1566574012
1,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4d71c7d495bc8eec74015ff9,"173/1, S.C. Road,",IN,Bangalore,India,Seshadripuram,2313,"[173/1, S.C. Road, (Seshadripuram), Bangalore ...","[{'label': 'display', 'lat': 12.98939463563121...",12.989395,77.572764,560020,Karnātaka,New Samrat Restaurant,v-1566574012
2,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4d01ec0e3be26dcb537475e4,J C Road,IN,Bangalore,India,NaN,2800,"[J C Road, Bangalore 560002, Karnātaka, India]","[{'label': 'display', 'lat': 12.95612026676884...",12.956120,77.580845,560002,Karnātaka,Mayuri Restaurant,v-1566574012
3,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4f12e573e4b06c588c0319a3,NaN,IN,Bangalore,India,NaN,2198,"[Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.98887515906547...",12.988875,77.573682,NaN,Karnātaka,Uttarkhand Restaurant,v-1566574012
4,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4c793a984dc1ef3bd7fa2c9e,"Chalukya Hotel, Racecourse Road",IN,Bangalore,India,NaN,722,"[Chalukya Hotel, Racecourse Road, Bangalore, K...","[{'label': 'display', 'lat': 12.98463439463957...",12.984634,77.587782,NaN,Karnātaka,Samrat Restaurant,v-1566574012


### Clean Data frame

In [35]:
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Ujwal Bar & Restaurant,Indian Restaurant,"402-A, Millers Road, Vasanthnagar,",IN,Bangalore,India,NaN,1504,"[402-A, Millers Road, Vasanthnagar,, Bangalore...","[{'label': 'display', 'lat': 12.99227960861634...",12.992280,77.594473,560052,Karnātaka,4b6ee375f964a52029cf2ce3
1,New Samrat Restaurant,Indian Restaurant,"173/1, S.C. Road,",IN,Bangalore,India,Seshadripuram,2313,"[173/1, S.C. Road, (Seshadripuram), Bangalore ...","[{'label': 'display', 'lat': 12.98939463563121...",12.989395,77.572764,560020,Karnātaka,4d71c7d495bc8eec74015ff9
2,Mayuri Restaurant,Indian Restaurant,J C Road,IN,Bangalore,India,NaN,2800,"[J C Road, Bangalore 560002, Karnātaka, India]","[{'label': 'display', 'lat': 12.95612026676884...",12.956120,77.580845,560002,Karnātaka,4d01ec0e3be26dcb537475e4
3,Uttarkhand Restaurant,Indian Restaurant,NaN,IN,Bangalore,India,NaN,2198,"[Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.98887515906547...",12.988875,77.573682,NaN,Karnātaka,4f12e573e4b06c588c0319a3
4,Samrat Restaurant,Indian Restaurant,"Chalukya Hotel, Racecourse Road",IN,Bangalore,India,NaN,722,"[Chalukya Hotel, Racecourse Road, Bangalore, K...","[{'label': 'display', 'lat': 12.98463439463957...",12.984634,77.587782,NaN,Karnātaka,4c793a984dc1ef3bd7fa2c9e


In [37]:
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Restaurant_dataframe2

,name,categories,address,crossStreet,lat,lng,postalCode,state
0,Ujwal Bar & Restaurant,Indian Restaurant,"402-A, Millers Road, Vasanthnagar,",NaN,12.992280,77.594473,560052,Karnātaka
1,New Samrat Restaurant,Indian Restaurant,"173/1, S.C. Road,",Seshadripuram,12.989395,77.572764,560020,Karnātaka
2,Mayuri Restaurant,Indian Restaurant,J C Road,NaN,12.956120,77.580845,560002,Karnātaka
3,Uttarkhand Restaurant,Indian Restaurant,NaN,NaN,12.988875,77.573682,NaN,Karnātaka
4,Samrat Restaurant,Indian Restaurant,"Chalukya Hotel, Racecourse Road",NaN,12.984634,77.587782,NaN,Karnātaka
5,RR Restaurant,Indian Restaurant,Church Street,NaN,12.977999,77.609015,NaN,Karnātaka
6,Eden Park Restaurant,Indian Restaurant,"Next to Hindu Office, Off Infantry Road",NaN,12.983456,77.598707,560052,Karnātaka
7,Aathithya Restaurant,None,149 Infantry Road,NaN,12.983956,77.593435,NaN,Karnataka State
8,Hotel Shalimar Bar and Restaurant,Bar,322/1 CMH Road,Indiranagar,12.978253,77.637368,560038,Karnātaka
9,Imperial Restaurant,Indian Restaurant,Millers Road,NaN,12.991150,77.593837,NaN,Karnātaka


In [38]:
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,crossStreet,lat,lng,postalCode,state
1,New Samrat Restaurant,Indian Restaurant,"173/1, S.C. Road,",Seshadripuram,12.989395,77.572764,560020,Karnātaka
8,Hotel Shalimar Bar and Restaurant,Bar,322/1 CMH Road,Indiranagar,12.978253,77.637368,560038,Karnātaka
18,Nisarga Garden Restaurant,Indian Restaurant,Infantry Road,Opp. Commissioner's Office,12.983332,77.594431,560001,Karnātaka
20,All Rounder Restaurant,Chinese Restaurant,KSCA,MG Road,12.979494,77.599570,500001,Karnātaka
22,Imperial Restaurant,Indian Restaurant,"#42/5, Opp. Shivajinagar Bus Stand",Central Street,12.982176,77.603279,560005,Karnātaka
23,Savoury Restaurant,Middle Eastern Restaurant,Mosque road,Frazer Town,12.999504,77.615887,560005,Karnātaka


## Automotive Shop

In [39]:
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WTYXTX1VKJMLXOJPLAD1IJKLASAMY20FDVGFA1Z1X4XTYAHF&client_secret=XV3U42TL5SPPKSLA4KYBIKK3JW3FRLEJDZJJNMQCXS31ZY5J&ll=12.9791198,77.5912997&v=20180604&query=Shopping&radius=1000&limit=30'

In [40]:
sresults = requests.get(url).json()


In [41]:
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d124951735', 'name': 'A...",False,5632ec65498eb65f4c43e98e,BatteryBhai.com,IN,Bangalore,India,305,"[BatteryBhai.com, Bangalore 560001, Karnātaka,...","[{'label': 'display', 'lat': 12.97846402705198...",12.978464,77.594032,560001,Karnātaka,Best Car Battery Online Shopping Store,v-1566574170


### Clean dataframe

In [42]:
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Best Car Battery Online Shopping Store,Automotive Shop,BatteryBhai.com,IN,Bangalore,India,305,"[BatteryBhai.com, Bangalore 560001, Karnātaka,...","[{'label': 'display', 'lat': 12.97846402705198...",12.978464,77.594032,560001,Karnātaka,5632ec65498eb65f4c43e98e


In [45]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Best Car Battery Online Shopping Store,Automotive Shop,BatteryBhai.com,12.978464,77.594032,560001,Karnātaka


In [47]:
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Automotive Shop']
df_Shopping

,name,categories,address,lat,lng,postalCode,state
0,Best Car Battery Online Shopping Store,Automotive Shop,BatteryBhai.com,12.978464,77.594032,560001,Karnātaka


## Visualization of Hotel, PG services and Automotive shops

In [48]:
hotel_neighbourhood_df = pd.concat([df_hotel, df_PG, df_Shopping], ignore_index=True)
hotel_neighbourhood_df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,address,categories,crossStreet,lat,lng,name,postalCode,state
0,"3, Raj Bhavan Rd",Hotel,NaN,12.981980,77.594564,24 Carats,560001,Karnātaka
1,"5th Main, 5th Cross",Residential Building (Apartment / Condo),BTM 2nd Stage,12.914719,77.605568,Paying Guest,560076,Karnātaka
2,#60,Residential Building (Apartment / Condo),"4th Cross, Kanaka Nagar",13.034220,77.607800,Paying Guest Accommodation,560032,Karnātaka
3,30-103/1,Hostel,"Hennur 3rd cross, Kalyan Nagar",13.029033,77.635450,Sunrise Paying Guest,560043,Karnātaka
4,342 6th Cross KR Layout,Residential Building (Apartment / Condo),6th Phase JP Nagar,12.903570,77.581674,atithishala Paying Guest,560078,Karnātaka
5,"16, Raja Ram Mohan Roy Road,",Boarding House,Sampangi Rama Nagara,12.966786,77.595310,Lic Vip Guesthouse,560027,Karnātaka
6,"No 2814, Hal 2nd Stage, 18th A Main",Motel,"5th B Cross, Old Airport Rd",12.974301,77.645455,Cosmos Guest Homes,560008,Karnātaka
7,BatteryBhai.com,Automotive Shop,NaN,12.978464,77.594032,Best Car Battery Online Shopping Store,560001,Karnātaka


In [52]:
# Generate map to visualize hotel neighbourhood including automotive shop and PG 
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

## Visualization of TechPark, Restaurant and PG services

In [50]:
park_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_PG,], ignore_index=True)
park_neighbourhood_df

,name,categories,address,crossStreet,lat,lng,postalCode,state
0,Manyata Embassy Business Park,Corporate Amenity,"Outer Ring Road, Nagavara",Nagavara,13.041540,77.619803,560 045,Karnātaka
1,New Samrat Restaurant,Indian Restaurant,"173/1, S.C. Road,",Seshadripuram,12.989395,77.572764,560020,Karnātaka
2,Hotel Shalimar Bar and Restaurant,Bar,322/1 CMH Road,Indiranagar,12.978253,77.637368,560038,Karnātaka
3,Nisarga Garden Restaurant,Indian Restaurant,Infantry Road,Opp. Commissioner's Office,12.983332,77.594431,560001,Karnātaka
4,All Rounder Restaurant,Chinese Restaurant,KSCA,MG Road,12.979494,77.599570,500001,Karnātaka
5,Imperial Restaurant,Indian Restaurant,"#42/5, Opp. Shivajinagar Bus Stand",Central Street,12.982176,77.603279,560005,Karnātaka
6,Savoury Restaurant,Middle Eastern Restaurant,Mosque road,Frazer Town,12.999504,77.615887,560005,Karnātaka
7,Paying Guest,Residential Building (Apartment / Condo),"5th Main, 5th Cross",BTM 2nd Stage,12.914719,77.605568,560076,Karnātaka
8,Paying Guest Accommodation,Residential Building (Apartment / Condo),#60,"4th Cross, Kanaka Nagar",13.034220,77.607800,560032,Karnātaka
9,Sunrise Paying Guest,Hostel,30-103/1,"Hennur 3rd cross, Kalyan Nagar",13.029033,77.635450,560043,Karnātaka


In [51]:
# Generate map to visualize park neighbourhood including Restaurant and PG
park_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(park_neighbourhood_df['lat'], park_neighbourhood_df['lng'], 
                                           park_neighbourhood_df['name'], park_neighbourhood_df['categories'],\
                                               park_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(park_map)  
    
park_map